In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

class SwarmAgent:
    def __init__(self, agent_id, position):
        self.agent_id = agent_id
        self.position = position

class Discriminator(nn.Module):
    def __init__(self, input_size):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

class AIRL:
    def __init__(self, num_agents, environment_size, expert_trajectories, num_epochs, learning_rate):
        self.num_agents = num_agents
        self.environment_size = environment_size
        self.expert_trajectories = expert_trajectories
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate

        self.discriminator = Discriminator(environment_size[0] * environment_size[1])
        self.generator = nn.Sequential(
            nn.Linear(environment_size[0] * environment_size[1], 64),
            nn.ReLU(),
            nn.Linear(64, environment_size[0] * environment_size[1]),
            nn.Tanh()
        )
        self.optimizer_d = optim.Adam(self.discriminator.parameters(), lr=learning_rate)
        self.optimizer_g = optim.Adam(self.generator.parameters(), lr=learning_rate)
        self.criterion = nn.BCELoss()

    def train(self):
        expert_states = self.get_state_trajectories(self.expert_trajectories)
        expert_labels = torch.ones(len(expert_states)).unsqueeze(1)

        for epoch in range(self.num_epochs):
            self.optimizer_d.zero_grad()

            # Train discriminator with expert data
            expert_output = self.discriminator(expert_states)
            loss_expert = self.criterion(expert_output, expert_labels)
            loss_expert.backward()

            # Train discriminator with generated data
            generated_states = self.generate_states(len(expert_states))
            generated_labels = torch.zeros(len(generated_states)).unsqueeze(1)
            generated_output = self.discriminator(generated_states.detach())
            loss_generated = self.criterion(generated_output, generated_labels)
            loss_generated.backward()

            self.optimizer_d.step()

            # Train generator
            self.optimizer_g.zero_grad()
            generated_output = self.discriminator(generated_states)
            loss_generator = self.criterion(generated_output, expert_labels)
            loss_generator.backward()
            self.optimizer_g.step()

            if epoch % 10 == 0:
                print(f"Epoch: {epoch}, Discriminator Loss: {loss_expert.item() + loss_generated.item()}, Generator Loss: {loss_generator.item()}")

    def get_state_trajectories(self, trajectories):
        states = []

        for trajectory in trajectories:
            agent_positions = [agent.position for agent in trajectory]
            state = np.zeros(self.environment_size)
            for agent_position in agent_positions:
                state[tuple(agent_position)] += 1
            states.append(state.flatten())

        return torch.tensor(states, dtype=torch.float)

    def generate_states(self, num_states):
        noise = torch.randn(num_states, self.environment_size[0] * self.environment_size[1])
        generated_states = self.generator(noise)
        return generated_states

# Usage example
num_agents = 10
environment_size = (10, 10)
num_epochs = 1000
learning_rate = 0.001

swarm_agents = []
for i in range(num_agents):
    position = np.random.randint(0, environment_size[0]), np.random.randint(0, environment_size[1])
    agent = SwarmAgent(i, position)
    swarm_agents.append(agent)

expert_trajectories = [swarm_agents]  # List of expert agent trajectories

airl = AIRL(num_agents, environment_size, expert_trajectories, num_epochs, learning_rate)
airl.train()


<ipython-input-1-62d2e131f6c3>:84: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.tensor(states, dtype=torch.float)


Epoch: 0, Discriminator Loss: 1.3538991808891296, Generator Loss: 0.7895625233650208
Epoch: 10, Discriminator Loss: 1.162130892276764, Generator Loss: 0.7539633512496948
Epoch: 20, Discriminator Loss: 1.039968490600586, Generator Loss: 0.7292135953903198
Epoch: 30, Discriminator Loss: 0.9167066216468811, Generator Loss: 0.7773917317390442
Epoch: 40, Discriminator Loss: 1.0213245451450348, Generator Loss: 0.6159793734550476
Epoch: 50, Discriminator Loss: 0.7333255410194397, Generator Loss: 0.9421995878219604
Epoch: 60, Discriminator Loss: 0.6393655091524124, Generator Loss: 1.162429690361023
Epoch: 70, Discriminator Loss: 1.1317084431648254, Generator Loss: 0.6414823532104492
Epoch: 80, Discriminator Loss: 1.2636996507644653, Generator Loss: 0.640957236289978
Epoch: 90, Discriminator Loss: 1.5276044011116028, Generator Loss: 0.5240001678466797
Epoch: 100, Discriminator Loss: 1.1340372562408447, Generator Loss: 0.9751554131507874
Epoch: 110, Discriminator Loss: 1.4788191318511963, Genera

The code starts by defining a SwarmAgent class representing an agent in the swarm. Each agent has an agent ID and a position in the environment.

The Discriminator class is defined as a neural network with two fully connected layers and a sigmoid activation function.

The AIRL class represents the Adversarial Inverse Reinforcement Learning algorithm. It takes the number of agents, environment size, expert trajectories, number of training epochs, and learning rate as input.

The AIRL class contains a discriminator and a generator. The discriminator estimates the reward function, and the generator generates states.

The train method trains the discriminator and generator using the expert trajectories. It iterates for the specified number of epochs.

The discriminator is trained using both expert and generated data. Expert trajectories are labeled as 1, and generated trajectories are labeled as 0.

The generator is trained to generate states that fool the discriminator by minimizing the generator loss.

The get_state_trajectories method converts agent trajectories into state trajectories. Each state is represented by a flattened array, where each element represents the density of agents in that position.

The generate_states method generates states using the generator model.

In the usage example, a swarm of agents is created with random positions in the environment.

Expert trajectories are defined as a list of agent trajectories. In this example, only a single expert trajectory is used.

An instance of AIRL is created, passing the necessary parameters.

The train method is called to train the discriminator and generator.